In [1]:
# ESTUDO DO PROBLEMA DE ESCALONAMENTO DE HORÁRIOS E ALGORITMOS GENÉTICOS

# Cada programa terá vários cursos
# Cada programa terá n semestres
# Cada curso pertencerá à um semestre especifico
# Cada semestre não poderá superar mais do que 10 aulas semanais no horario

# AG       -> Programa{ Cursos, Profs, Salas } : Schedule
# Programa -> {Horario Semanal para cada o programa com n semestres}

# *OBS1: Haverá "ataques ou choque de horarios" em um schedule para programa com n semestres (ataque entre semestres)
# *OBS2: Após a geração de um schedule pra um programa, guardar restrições desse programa para aplicar em outro, caso use os mesmos requisitos (salas e professores)


# ------------ AG ------------
# ___Code___
# Laço:
# - Popular
# - Fitness
# - Testar -> (Sol, S/ Sol, Evoluir)
# Fim

# Evoluir:
# - Seleção
# - Cruzamento
# - Mutação
# __________

# Conceitos:
# • Definir o que é cromossomo
# • Teste de Meta: Sim, se algum cromossomo tem fitness = 1. Não, caso o contrário.
# • Fitness: (1 - Erro/MaxErro)
#                  (Acerto/MaxAcerto)
# • Seleção: Roleta Viciada (Gera 2*n casais de cromossomos. Partição da roleta é baseada no valor Fitness)
# • Cruzamento: Cruzar casais gerados na Seleção. Ao cruzar, é pegada informação do macho e da fêmea, e são gerados 2 filhos melhores geneticamente.
# • Mutação: Mutar gene de cromossomos aleatoriamente

In [2]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy
import itertools as it

In [3]:
### Data Structure for the Program Schedule Problem ###

# Class Course
class Course:
    _id = 0
    def __init__(self, name, classes, semester, teacherId=-1):
        Course._id     = Course._id +1
        self.id        = Course._id
        self.name      = name
        self.classes   = classes
        self.ammo      = classes
        self.semester  = semester
        self.teacherId = teacherId
        
    def __repr__(self):
        return 'Course {}: {} ({} classes, {}o semester, {} teacherId)'.format(self.id, self.name, self.classes, self.semester, self.teacherId)

# Class Teacher
class Teacher:
    _id = 0
    def __init__(self, name):
        Teacher._id  = Teacher._id +1
        self.id      = Teacher._id
        self.name    = name
        
    def __repr__(self):
        return 'Teacher {}: {}'.format(self.id, self.name)
    
# Class Classroom
class Classroom:
    _id = 0
    def __init__(self, name):
        Classroom._id = Classroom._id +1
        self.id       = Classroom._id
        self.name     = name
    
    def __repr__(self):
        return 'Classroom {}: {}'.format(self.id, self.name)

# Class Program
class Program:
    def __init__(self, name, semesters, courses=[], classrooms=[], teachers=[]):
        self.name       = name
        self.semesters  = semesters
        self.courses    = courses
        self.classrooms = classrooms
        self.teachers   = teachers
    
    def getCoursesBySemester(self, semester):
        coursesSemester = []
        for i in self.courses:
            if i.semester is semester and i.ammo > 0:
                coursesSemester.append(i)
        return coursesSemester
    
    def reloadCoursesAmmo(self):
        for i in self.courses:
            i.ammo = i.classes
            
    def addCourse(self, course):
        try:
            if course not in self.courses:
                self.courses.append(course)
        except:
            return
    
    def removeCourse(self, course):
        try:
            self.courses.remove(course)
        except:
            return
    
    def addTeacher(self, teacher):
        try:
            if teacher not in self.teachers:
                self.teachers.append(teacher)
        except:
            return
    
    def removeTeacher(self, teacher):
        try:
            self.teachers.remove(teacher)
        except:
            return
        
    def addClassroom(self, classroom):
        try:
            if classroom not in self.classrooms:
                self.classrooms.append(classroom)
        except:
            return
    
    def removeClassroom(self, classroom):
        try:
            self.classrooms.remove(classroom)
        except:
            return
        
    def __repr__(self):
        return 'Program: {} ({} semesters, {} courses, {} classrooms, {} teachers)'.format(self.name, self.semesters, len(self.courses), len(self.classrooms), len(self.teachers))

#Testes
co1 = Course('Calculo I', 2, 1)
co2 = Course('Calculo II', 2, 2)
co3 = Course('Algebra Linear', 2, 2)
te1 = Teacher('Enio Romagnome')
te2 = Teacher('Roberto Carlos')
cl1 = Classroom('BC-23')
cl2 = Classroom('BC-08')
co1.teacherId = te1.id
co2.teacherId = te2.id
co3.teacherId = te2.id

p1 = Program('EngComp', 10)
p1.addCourse(co1)
p1.addCourse(co2)
p1.addCourse(co3)
p1.addTeacher(te1)
p1.addTeacher(te2)
p1.addClassroom(cl1)
p1.addClassroom(cl2)
print(p1)

Program: EngComp (10 semesters, 3 courses, 2 classrooms, 2 teachers)


In [4]:
class Schedule:
    def __init__(self, program):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.schedule = list([[[0 for z in range(2)] for j in range(5)] for i in range(program.semesters)])
        self.generateRandomSchedule(program)
        program.reloadCoursesAmmo()
        
    def generateRandomSchedule(self, program):
        for i in range(1, program.semesters+1):
            history = []
            while(True):
                courses = program.getCoursesBySemester(i)
                if(courses == []):
                    break
                randomCourse = random.choice(courses)
                program.courses[program.courses.index(randomCourse)].ammo-=1
                #Put class on a random place
                while(True):
                    random1 = random.randint(1,5)
                    random2 = random.randint(1,2)
                    if (random1,random2) not in history:
                        history.append((random1,random2))
                        break
                self.schedule[i-1][random1-1][random2-1] = randomCourse.id
        return True
    
    def __repr__(self):
        strOut = '|  '
        for i in self.weekName:
            strOut += i + '  '
        strOut += '|\n'
        for i in range(len(self.schedule)):
            strOut += '|---------------------------|\n'
            if(i+1>=10):
                strOut += '|        ' + 'Semester ' + str(i+1) + '        |\n'
            else:
                strOut += '|        ' + 'Semester ' + str(i+1) + '         |\n'
            strOut += '|---------------------------|\n'
            strOut += '|  '
            for j in self.schedule[i]:
                if(j[0]>=10):
                    strOut += ''  + str(j[0]) + '   '
                else: 
                    strOut += ' ' + str(j[0]) + '   '
            strOut += '| AB\n'
            strOut += '|  '
            for j in self.schedule[i]:
                if(j[0]>=10):
                    strOut += ''  + str(j[1]) + '   '
                else: 
                    strOut += ' ' + str(j[1]) + '   '
            strOut += '| CD\n'
        return strOut

#Testes
s = Schedule(p1)
print(s)


|  Seg  Ter  Qua  Qui  Sex  |
|---------------------------|
|        Semester 1         |
|---------------------------|
|   0    0    0    0    1   | AB
|   0    0    0    0    1   | CD
|---------------------------|
|        Semester 2         |
|---------------------------|
|   3    0    0    0    0   | AB
|   0    0    3    2    2   | CD
|---------------------------|
|        Semester 3         |
|---------------------------|
|   0    0    0    0    0   | AB
|   0    0    0    0    0   | CD
|---------------------------|
|        Semester 4         |
|---------------------------|
|   0    0    0    0    0   | AB
|   0    0    0    0    0   | CD
|---------------------------|
|        Semester 5         |
|---------------------------|
|   0    0    0    0    0   | AB
|   0    0    0    0    0   | CD
|---------------------------|
|        Semester 6         |
|---------------------------|
|   0    0    0    0    0   | AB
|   0    0    0    0    0   | CD
|---------------------------|
|   